In [122]:
import tensorflow as tf
from keras.layers import Input, Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D, concatenate, Activation
from keras.applications.mobilenet import MobileNet as Net

from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
import cv2
import numpy as np
import os
from tqdm import tqdm
from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

ALPHA = 0.3  # Triplet Loss Parameter

from keras.layers import Input,Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D,concatenate,Activation
from keras.applications.xception import Xception as Net
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model

import pydot
import graphviz
import time
import glob


import matplotlib.pyplot as plt

In [6]:
def img_to_array(image):
    img = cv2.resize(image, (128, 128)) 
    x_train = np.array([img])
    return x_train

In [7]:
def img_path_to_array(image_path):
    img1 = cv2.imread(image_path, 1)
    return img_to_array(img1)

In [54]:
def prepare_test_arrays(location):
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob(location):
        identity = os.path.splitext(os.path.basename(file))[0].strip('NFI-')
        #print (identity)
        database[identity] = img_path_to_array(file)

    return database

In [55]:
test_genuine_arrays = prepare_test_arrays('/Users/ochhab3/Documents/ojaswinich_github/signature_verification/images/sigcomp2009/test/genuines/*')

In [56]:
test_forgery_arrays = prepare_test_arrays('/Users/ochhab3/Documents/ojaswinich_github/signature_verification/images/sigcomp2009/test/forgeries/*')

In [65]:
all_ids = np.arange(1, 101)
missing_ids = np.array([5, 13, 25, 32, 34, 36, 38, 40, 48, 50, 52, 57, 60, 61, 65, 76, 78, 81, 82, 87, 95])
ids_available = np.array(list((set(all_ids)-set(missing_ids))))

In [49]:
test_database_with_id = {}

for i in ids_available:
    
    if (i < 10):
        image_id = '00'+str(i)+'0100'+str(i)
    elif (9 < i < 100):   
        image_id = '0'+str(i)+'010'+str(i)
    else:
        image_id = str(i)+'01'+str(i)
        
    test_database_with_id[i] = test_genuine_embeddings[image_id]
    

In [104]:
pairwise_gen_ids = []

for i in ids_available:
    for j in range(2, 13):
        
        if (i < 10 and j < 10):
            a='00'+ str(i)+'0100'+str(i)
            p='00'+ str(i)+'0'+str(j)+'00'+str(i)
        elif (i < 10 and j > 9):
            a='00'+ str(i)+'0100'+str(i)
            p='00'+ str(i)+str(j)+'00'+str(i)
        elif (9 < i < 100 and j < 10):
            a='0'+str(i)+'010'+str(i)
            p='0'+ str(i)+'0'+str(j)+'0'+str(i)
        elif (9 < i < 100 and j > 9):
            a='0'+str(i)+'010'+str(i)
            p='0'+ str(i)+str(j)+'0'+str(i)
        elif (i > 99 and j < 10):
            a=str(i)+'01'+str(i)
            p=str(i)+'0'+str(j)+str(i)
        elif (i > 99 and j > 9):
            a=str(i)+'01'+str(i)
            p=str(i)+str(j)+str(i)
        pair_gen = (a,p)        
        pairwise_gen_ids.append(pair_gen)        


In [113]:
pairwise_sforg_ids = []

for i in ids_available:
    for f in range(3, 101):
        for n in range(1, 7):
            if (i<10 and f<10):
                a='00'+ str(i)+'0100'+str(i)
                n='00'+ str(f)+'0'+str(n)+'00'+str(i)
            elif (i<10 and 9<f<100):
                a= '00'+ str(i)+'0100'+str(i)
                n= '0'+ str(f)+'0'+str(n)+'00'+str(i)
            elif (i<10 and f>99):
                a= '00'+ str(i)+'0100'+str(i)
                n=  str(f)+'0'+str(n)+'00'+str(i)
            elif (9<i<100 and f<10):
                a='0'+ str(i)+'010'+str(i)
                n='00'+ str(f)+'0'+str(n)+'0'+str(i)
            elif(9<i<100 and 9<f<100):
                a='0'+ str(i)+'010'+str(i)
                n='0'+ str(f)+'0'+str(n)+'0'+str(i)
            elif(9<i<100 and f>99):
                a='0'+ str(i)+'010'+str(i)
                n= str(f)+'0'+str(n)+'0'+str(i)
            elif(i>99 and f<10):
                a= str(i)+'01'+str(i)
                n='00'+ str(f)+'0'+str(n)+str(i)
            elif(i>99 and 9<f<100):
                a= str(i)+'01'+str(i)
                n='0'+ str(f)+'0'+str(n)+str(i)
            elif(i>99 and f>99):
                a= str(i)+'01'+str(i)
                n= str(f)+'0'+str(n)+str(i)
            
            sforg_ids=(a,n)
            pairwise_sforg_ids.append(sforg_ids)

In [118]:
pairwise_ugen_ids = []

for ai in ids_available:
    for ui in ids_available:
        if (ai!=ui and 9<ai<100 and 9<ui<100):
            a='0'+str(ai)+'010'+str(ai)
            u='0'+str(ui)+'010'+str(ui)
            pair_ugen = (a,u)        
            pairwise_ugen_ids.append(pair_ugen)      


In [ ]:
#Framework defination steps
#1.Input----> id and image
#2.Get the image embedding from our previously trained model
#3.It will check the id in the database key.
#4.If absent,it will save the embedding in the database against the key
#5.If present, it will compare the embedding with the pre-saved embedding corresponding to the same id in the database 
#and declare as genuine or forge based on the probability obtained from our Logistic regression model.


In [ ]:
#input_id is the input key
#input_img is the corresponding image

In [123]:
#Step1----Getting test image
def img_path_to_array(image_path):
    img1 = cv2.imread(image_path, 1)
    return img_to_array(img1)

In [124]:
def get_triple_embedding(image_path):
    input_img= img_path_to_array(image_path)
    img_embed=intermediate_layer_model_anchor.predict([input_img,input_img,input_img])
    return img_embed

In [128]:
#database is a dictionary with the ids and embedding
def check_if_in_db(database,image_id):
    if image_id in database.keys():
        output='present'
    else :
        output='absent'
    return output
        

In [134]:
def add_img_to_db(database,image_id,image_embed):
    database['image_id']=image_embed
    return database

In [135]:
def predict_img_status(database,image_id, image_embed,log_model):
    anchor_embed=database['image_id']
    test=np.linalg.norm(anchor_embed-image_embed)
    pred=log_model.predict(test)
    if (pred==1):
        output='genuine'
    else :
        output='forged'
    return output

In [ ]:
def final_framework(database, image_path, image_id):
    # get image embedddings
    image_embed=get_triple_embedding(image_path)
    # check if image id is present in the database
    check=check_if_in_db(database,image_id)
    # if image is not present, then add to database dictionary
    if (check=='absent'):
        database=add_img_to_db(database,image_id,image_embed)
        final_output='Added the new ID to the database with embedding.'
    else:
        prediction_output=predict_img_status(database,image_id, image_embed)
        if (prediction_output=='genuine'):
            final_output='This is a genuine signature.'
        else:
            final_output='This is a forged signature.'
    return final_output  